In [1]:
import os, sys
sys.path.append('..')

import io
import torch
import numpy as np
import numpy.random as npr
import hydra
import dill
from tqdm import tqdm

import plotly.io as pio
pio.renderers.default = 'jupyterlab'
from scipy.interpolate import griddata
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import imageio

from irrep_actions.gym_util.multistep_wrapper import MultiStepWrapper
from irrep_actions.env.pusht.pusht_keypoints_env import PushTKeypointsEnv
from irrep_actions.workflow.base_workflow import BaseWorkflow
from irrep_actions.utils import mcmc

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# CH SO2 Lmax 5
#checkpoint = '../data/outputs/2024.01.30/16.43.31_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/epoch=0350-test_mean_score=0.866.ckpt'

# CH SO2 Lmax 3
#checkpoint = '../data/outputs/2024.01.30/06.16.01_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/epoch=0400-test_mean_score=0.975.ckpt'

# CH skip
# checkpoint = '../data/outputs/2024.01.30/19.08.21_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/latest.ckpt'

#checkpoint = '../data/outputs/2024.02.01/21.53.16_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/epoch=0800-test_mean_score=0.740.ckpt'
checkpoint = '../data/outputs/2024.02.02/21.28.28_train_so2_harmonic_implicit_policy_pusht_lowdim/checkpoints/epoch=0600-test_mean_score=0.824.ckpt'

payload = torch.load(open(checkpoint, 'rb'), pickle_module=dill)
device = torch.device('cuda:3')

In [3]:
cfg = payload['config']
cls = hydra.utils.get_class(cfg._target_)

workflow = cls(cfg)
workflow: BaseWorkflow
workflow.load_payload(payload, exclude_keys=None, include_keys=None)

policy = workflow.model
policy = policy.to(device)
policy.energy_model.eval()

ImplicitPolicy(
  (normalizer): LinearNormalizer(
    (params_dict): ParameterDict(
        (obs): Object of type: ParameterDict
        (action): Object of type: ParameterDict
      (obs): ParameterDict(
          (offset): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:3)]
          (scale): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:3)]
          (input_stats): Object of type: ParameterDict
        (input_stats): ParameterDict(
            (max): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:3)]
            (mean): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:3)]
            (min): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:3)]
            (std): Parameter containing: [torch.cuda.FloatTensor of size 38 (cuda:3)]
        )
      )
      (action): ParameterDict(
          (offset): Parameter containing: [torch.cuda.FloatTensor of size 2 (cuda:3)]
          (scale): Parameter containing: [torch.c

In [11]:
def eval(num_eps, start_seed=100000, num_disp = 100, num_rot=360, sample_act=False, temp=1.0, vid_out=True):
    pbar = tqdm(total=num_eps)
    max_rewards = [0] * num_eps
    for eps in range(num_eps):
        env = MultiStepWrapper(
            PushTKeypointsEnv(render_action=False, random_goal_pose=False), 
            n_obs_steps=2, 
            n_action_steps=1,
            max_episode_steps=300
        )
        env.seed(start_seed + eps)
        
        obs = env.reset()
        d = False
        B = 1
        plots = list()
        
        while not d:
            Do = obs.shape[-1] // 2 
            obs = torch.from_numpy(obs[:, :Do].astype(np.float32).reshape(B,2,-1,2)).to(device) # 1x2x19x2
            x_obs = (obs.reshape(1,38,2)[:,:,0] - 255.0)
            y_obs = (obs.reshape(1,38,2)[:,:,1] - 255.0) * -1.0
            new_d = torch.concatenate((x_obs.unsqueeze(-1), y_obs.unsqueeze(-1)), dim=-1).view(1, -1).view(1,2,19*2)
            nobs = policy.normalizer['obs'].normalize(new_d)
            
            action_stats = policy.get_action_stats()
            action_dist = torch.distributions.Uniform(
                low=action_stats["min"], high=action_stats["max"]
            )
            mag = torch.linspace(-1.0, 1.0, num_disp)
            mag = mag.view(1, -1).repeat(B, 1).view(B, -1, 1, 1).to(device)
            theta = torch.linspace(0, 2*np.pi, num_rot).to(device)
    
            with torch.no_grad():
                logits = policy.energy_model.get_energy_ball(nobs, mag).view(1, -1)
            action_probs = torch.softmax(logits/temp, dim=-1).view(1, num_disp, num_rot)

            if not sample_act:
                flat_indexes = action_probs.flatten(start_dim=-2).argmax(1)
                idx = [divmod(idx.item(), action_probs.shape[-1]) for idx in flat_indexes][0]
                #idxs = torch.argmax(action_probs, dim=-1).unsqueeze(-1)
            else:
                flat_indexes = torch.multinomial(action_probs.flatten(start_dim=-2), num_samples=1, replacement=True)
                idx = [divmod(idx.item(), action_probs.shape[-1]) for idx in flat_indexes][0]
                #idxs = torch.multinomial(action_probs, num_samples=1, replacement=True)
                
            #actions = actions[torch.arange(B).unsqueeze(-1), idxs].squeeze(1)
            actions = torch.tensor([mag[0,idx[0],0,0], theta[idx[1]]])
            if True:
                #action_probs = action_probs.view(B, num_disp, num_rot)
                max_disp = torch.max(action_probs, dim=-1)[0]
                E = action_probs[0,torch.argmax(max_disp, dim=1).item()].cpu().numpy()
          
            mag = policy.normalizer["action"].unnormalize(actions)[0]
            #theta = policy.normalizer["action"].unnormalize(actions)[:,:,1]
            theta = actions[1]
            x = mag * torch.cos(theta)
            y = mag * torch.sin(theta)
            actions = torch.concat([x.view(B,1), y.view(B,1)], dim=1).unsqueeze(1)
            
            if vid_out:
                img = env.render('human')
                
                f = plt.figure(figsize=(10,3))
                ax1 = f.add_subplot(111)
                ax2 = f.add_subplot(141, projection='polar')
                ax1.imshow(img)
                ax2.plot(np.linspace(0, 2*np.pi, E.shape[0]), E)
                #ax2.set_rmax(np.max(E))
                #ax2.set_rticks(np.round(np.linspace(np.min(E), np.max(E), 5), 3))
                ax2.set_rticks(list())
                ax2.grid(True)
                
                ax2.set_title(f"R={mag.item():.3f}", va="bottom")
                #plt.show()
                io_buf = io.BytesIO()
                f.savefig(io_buf, format='raw')
                io_buf.seek(0)
                img_arr = np.reshape(np.frombuffer(io_buf.getvalue(), dtype=np.uint8),
                             newshape=(int(f.bbox.bounds[3]), int(f.bbox.bounds[2]), -1))
                plots.append(img_arr)
                io_buf.close()
                plt.close() 
        
            x_act = actions[:,:,0]
            y_act = actions[:,:,1] * -1.0
            new_act = torch.concatenate((x_act, y_act), dim=-1).view(B,1,2)
            obs, r, d, _ = env.step(new_act.squeeze(0).cpu().numpy())
            max_rewards[eps] = max(r, max_rewards[eps])
        imageio.mimwrite(f'plots/lmax_5/{eps}.gif', plots)
        print(max_rewards[eps])
        pbar.update(1)
    return max_rewards

In [ ]:
max_rewards = eval(num_eps=50, start_seed=100000, num_disp=5000, num_rot=360, sample_act=True, temp=5.0)

  0%|                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
  2%|███▎                                                                                                                    

0.1958990038602754


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
  4%|██████▋                                                                                                                                                                 | 2/50 [01:45<42:20, 52.92s/it]

0.8459065264942757


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
  6%|██████████                                                                                                                                                              | 3/50 [02:38<41:31, 53.02s/it]

...

ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
 68%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                     | 34/50 [33:15<14:57, 56.11s/it]

0.1711466456166625


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
 70%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                  | 35/50 [34:09<13:49, 55.32s/it]

0.4150818300019138


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


In [9]:
np.mean(max_rewards)

0.8260005939202706

In [10]:
max_rewards

[1.0,
 0.8999437227148788,
 1.0,
 0.9333167118815405,
 1.0,
 1.0,
 0.9881503336402542,
 1.0,
 0.792496838510295,
 0.0,
 0.8325388219441212,
 0.9617743564861938,
 1.0,
 0.7757263225673964,
 0.9134389780544954,
 0.7360066930425169,
 1.0,
 0.9768370458660712,
 0.9019078554235633,
 0.8599903313257979,
 0.683881008174098,
 0.9846447002537959,
 1.0,
 0.8621217762309058,
 0.961200688135919,
 0.9051833174363605,
 1.0,
 0.8851059458888684,
 1.0,
 1.0,
 0.47687637197573873,
 0.5699421280452746,
 0.09381394709921152,
 0.11442276710517645,
 0.0,
 0.4612515016732053,
 0.8555479604620121,
 1.0,
 0.9032547365406352,
 0.9641816970202768,
 0.9494781024757821,
 0.8229473112242903,
 0.9384436450414524,
 1.0,
 0.6768513058087512,
 0.9985130037038712,
 0.7826100724142446,
 0.9209983928321643,
 0.9166313050143672,
 1.0]